In [ ]:
import pickle
import sys
from zoneinfo import ZoneInfo
sys.path.append("../")
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import boto3
from dotenv import load_dotenv
load_dotenv()
import datetime

from openbustools import spatial, standardfeeds

import plotly.express as px
import pandas as pd

from pathlib import Path
import fastsim as fsim


from openbustools import standardfeeds, spatial, plotting
from openbustools.drivecycle import trajectory

### Presentation Plots

In [ ]:
s3 = boto3.resource('s3')
bucket = s3.Bucket("gtfs-collection-kcm")
objs = list(bucket.objects.all())
available_files = [o.key[:10] for o in objs]
dates_list = [datetime.datetime.strptime(date, '%Y_%m_%d').date() for date in available_files]
kcm_max, kcm_min = max(dates_list).strftime('%Y-%m-%d'), min(dates_list).strftime('%Y-%m-%d')
kcm_min = "2020-08-01"

s3 = boto3.resource('s3')
bucket = s3.Bucket("gtfs-collection-nwy")
objs = list(bucket.objects.all())
available_files = [o.key[:10] for o in objs]
dates_list = [datetime.datetime.strptime(date, '%Y_%m_%d').date() for date in available_files]
nwy_max, nwy_min = max(dates_list).strftime('%Y-%m-%d'), min(dates_list).strftime('%Y-%m-%d')

s3 = boto3.resource('s3')
bucket = s3.Bucket("gtfs-collection-others")
objs = list(bucket.objects.all())
available_files = [o.key[:10] for o in objs]
dates_list = [datetime.datetime.strptime(date, '%Y_%m_%d').date() for date in available_files]
other_max, other_min = max(dates_list).strftime('%Y-%m-%d'), min(dates_list).strftime('%Y-%m-%d')

In [ ]:
df = pd.DataFrame([
    dict(Task="King County Metro (Seattle, US)", Start=kcm_min, Finish=kcm_max, Network="KCM"),
    dict(Task="AtB (Trondheim, Norway)", Start=nwy_min, Finish=nwy_max, Network="AtB"),
    dict(Task="40 Other International Cities", Start=other_min, Finish=other_max, Network="Other Cities"),
])

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Task", color="Network", width=1200, height=400)
fig.update_layout(yaxis_title="", xaxis_title="Collection Dates")
fig.update_yaxes(autorange="reversed")
fig.update_layout(showlegend=False)
fig.show()
fig.write_image("../plots/gtfsrt_collection_dates.png")

In [ ]:
cleaned_sources = pd.read_csv("../data/cleaned_sources.csv")

fleet_sizes = []
for i, row in cleaned_sources.iterrows():
    available = list(Path(f"../data/other_feeds/{row['uuid']}_realtime").glob("*.pkl"))
    if len(available) == 0:
        print(f"No data for {row['provider']}")
        fleet_sizes.append(411)
        continue
    data = pd.read_pickle(available[0])
    fleet_sizes.append(data.vehicle_id.nunique())

cleaned_sources['fleet_size'] = np.array(fleet_sizes)
cleaned_sources = cleaned_sources.dropna()

In [ ]:
fig = px.scatter_geo(cleaned_sources, lat="min_lat", lon="min_lon",
                     hover_name="provider",
                     projection="natural earth",
                     width=1400,
                     height=800
                     )
fig.update_layout(title="GTFS-RT Collection Networks", margin={"r":10,"t":0,"l":10,"b":0})
fig.show()
fig.write_image("../plots/gtfsrt_collection_locations.png")

### Use Data from FTA

In [ ]:
pmt_plot_df = pd.DataFrame({
    'Mode': ['Motor bus', 'Commuter bus', 'Light rail', 'Heavy rail', 'Trolley bus', 'Commuter rail', 'Demand response', 'Ferry boat', 'Other'],
    'val': [8243, 506, 1041, 7405, 56, 3707, 481, 268, 665]
}).sort_values('val', ascending=False)
# plot_df = pd.DataFrame({
#     'Mode': ['Motor bus', 'Commuter bus', 'Light rail', 'Heavy rail', 'Trolley bus', 'Commuter rail', 'Demand response', 'Ferry boat', 'Other'],
#     'pmt': [10112,750,1474,9802,78,5924,637,377,793]
# }).sort_values('pmt', ascending=False)
pmt_plot_df['Passenger Kilometers Traveled (Million-PKT)'] = pmt_plot_df['val']*1.60934
sns.barplot(pmt_plot_df, x='Mode', y='Passenger Kilometers Traveled (Million-PKT)', hue='Mode')
# Don't show x-axis labels
plt.xlabel('')
plt.xticks(rotation=45)
plt.tight_layout()

In [ ]:
cap_plot_df = pd.DataFrame({
    'Mode': ['Motor bus', 'Heavy rail', 'Commuter rail', 'Light rail'],
    'val': [5178012810, 8155523157, 4817819823, 4719319058]
})
cap_plot_df['Capital Expenses (Million-$)'] = cap_plot_df['val']/1000000
sns.barplot(cap_plot_df, x='Mode', y='Capital Expenses (Million-$)', hue='Mode')
# Don't show x-axis labels
plt.xlabel('')
plt.xticks(rotation=45)
plt.tight_layout()

In [ ]:
op_plot_df = pd.DataFrame({
    'Mode': ['Motor bus', 'Heavy rail', 'Commuter rail', 'Light rail'],
    'val': [23885799593, 9362186080, 4967053257, 2378641425]
})
op_plot_df['Operating Expenses (Million-$)'] = op_plot_df['val']/1000000
sns.barplot(op_plot_df, x='Mode', y='Operating Expenses (Million-$)', hue='Mode')
# Don't show x-axis labels
plt.xlabel('')
plt.xticks(rotation=45)
plt.tight_layout()

In [ ]:
plot_df = cap_plot_df.merge(pmt_plot_df, on='Mode')
plot_df['Capital $/PKT'] = plot_df['Capital Expenses (Million-$)'] / plot_df['Passenger Kilometers Traveled (Million-PKT)']
sns.barplot(plot_df, x='Mode', y='Capital $/PKT', hue='Mode')
# Don't show x-axis labels
plt.xlabel('')
plt.xticks(rotation=45)
plt.tight_layout()

In [ ]:
plot_df = op_plot_df.merge(pmt_plot_df, on='Mode')
plot_df['Operating $/PKT'] = plot_df['Operating Expenses (Million-$)'] / plot_df['Passenger Kilometers Traveled (Million-PKT)']
sns.barplot(plot_df, x='Mode', y='Operating $/PKT', hue='Mode')
# Don't show x-axis labels
plt.xlabel('')
plt.xticks(rotation=45)
plt.tight_layout()